Check out Kaggle at https://www.kaggle.com/c/conways-reverse-game-of-life-2020. The objective is to reverse Conway's Game of Life Board (25x25 cells) to learn its position $\delta$ steps ago ($1<=\delta<=5$). We experiment with Convolvers, CNN Encoder-Decoders, and Generative Adverserial Networks, while evaluating performance on custom decaying losses (MSE/Brier, Entropy, Hinge) as well as ROC.

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import collections
import gc
import pickle
import time
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Lambda, Flatten, Conv2DTranspose
from tensorflow.keras.layers import Dropout, GaussianNoise, Input, UpSampling2D, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model
import sklearn.model_selection
from copy import deepcopy
import sklearn.metrics

<h2>Data Preprocessing</h2>

In [2]:
start = time.time()
data = pd.read_csv("./train.csv")
X1 = data[data.columns[np.vectorize(lambda s: "stop" in s)(data.columns.values)]]
X1 = X1.values.reshape(data.shape[0],25,25,1)
delta = data["delta"].values
X2 = np.zeros((data.shape[0],25*25*5))
for i in range(data.shape[0]):
    X2[i,(delta[i]-1)::5] = 1
temp = data[data.columns[np.vectorize(lambda s: "start" in s)(data.columns.values)]].values
Y = np.zeros((data.shape[0],25*25*5))
for i in range(data.shape[0]):
    Y[i,(delta[i]-1)::5] = temp[i]
print("Preprocessing Exited in "+str(time.time()-start)+" seconds")
data.shape, X1.shape, X2.shape, Y.shape

Preprocessing Exited in 8.559165239334106 seconds


((50000, 1252), (50000, 25, 25, 1), (50000, 3125), (50000, 3125))

<h2> SECTION 1: Building the Discriminator